In [48]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np
import math
from datetime import timedelta
from statsmodels.tsa.vector_ar.var_model import VAR


In [56]:
df1 = pd.read_csv("out_without_nan_mood_target.csv")
df1['date'] = pd.to_datetime(df1['date'])

In [57]:
df = df1.dropna(subset = ['mood_mean','mood_mean_TARGET'] )

In [58]:
# Create test and train set
length_train = 1
index_test = np.random.choice(np.arange(len(df)), length_train, False)

test = df.iloc[index_test]
train = df.drop(df.index[index_test])

In [79]:
id_person = test.id.values[0]
date = test.date.values[0]

In [80]:
train_data_person = df.loc[(df['id'] == id_person) & (df['date'] < date)]
train_data_person_correct = train_data_person[["circumplex.valence_mean", 'mood_mean','mood_mean_TARGET']]

In [49]:
#fit the model
model = VAR(endog=train_data_person_correct)
model_fit = model.fit()

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [55]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=1)
prediction_mood_target = prediction[0][2]

In [82]:
def predictMoodPerson(row):
    id_person = row.id
    date = row.date
    
    #get train data
    train_data_person = df.loc[(df['id'] == id_person) & (df['date'] < date)]
    train_data_person_correct = train_data_person[["circumplex.valence_mean", 'mood_mean','mood_mean_TARGET']]
    
    #fit the model
    model = VAR(endog=train_data_person_correct)
    model_fit = model.fit()
    
    # make prediction on validation
    prediction = model_fit.forecast(model_fit.y, steps=1)
    prediction_mood_target = prediction[0][2]
    
    return prediction_mood_target    

In [75]:
# Create empty df to save solutions
solutions = pd.DataFrame(data={'id': [], 'date': [], 'mood_mean_target': [], 'predicted_mood_mean_target':[]})

In [76]:
# Find for every row in test set the predicted mood
for index, row in test.iterrows():
    prediction_mood_target = predictMoodPerson(row)
    
    #add solution to solution df
    solutions = solutions.append({'id': id_person, 'date': date, 'mood_mean_target': row.mood_mean_TARGET, 
                                  'predicted_mood_mean_target': prediction_mood_target}, ignore_index=True)

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [77]:
solutions

,id,date,mood_mean_target,predicted_mood_mean_target
0,AS14.26,2014-04-03,7.2,7.1692
